In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('test.csv',delimiter=';')

In [95]:
def checkmissingvalue(df):
    missing = []
    total = []
    for x in df.columns:
        missing.append(df[x].isnull().sum())
        total.append(len(df[x]))
    daftar = pd.DataFrame([missing,total])  
    daftar.loc[3,:] = daftar.apply(lambda x: x[0] / x[1])
    daftar.columns = df.columns
    daftar.index = ['TotalMissingVal','TotalVal','%MissingVal']
    return daftar.convert_dtypes()

In [96]:
checkmissingvalue(df)

,a,b,c,d
TotalMissingVal,2.0,1.0,0,2.0
TotalVal,8.0,8.0,8,8.0
%MissingVal,0.25,0.125,0,0.25


In [99]:
def replacemissingvalue(df, dictt):
    for x in dictt:
        if dictt[x] == 'mean':
            df[x].fillna(df[x].mean(), inplace=True)
        else:
            df[x].fillna(dictt[x], inplace=True)
    return df.convert_dtypes()
            

In [101]:
replacemissingvalue(df,{'a' : 'mean','b' : 'p','d':'x'})

,a,b,c,d
0,1.0,p,4,5.0
1,23.0,2.0,4,5.0
2,4.0,3.0,4,5.0
3,5.0,3.0,4,x
4,6.0,3.0,4,5.0
5,7.833333,3.0,4,x
6,7.833333,3.0,4,5.0
7,8.0,3.0,5,5.0


In [113]:
def numericategoric(df):
    num = len(df._get_numeric_data().columns)
    cat = len(df.columns) - num
    print("TotalNumericalData = " + str(num))
    print("TotalCategoricalData = " + str(cat))
    print("Numerical = " + str(list(df._get_numeric_data().columns )))
    print("Categorical = " + str(list(df.drop(df._get_numeric_data().columns, axis=1).columns)))

In [114]:
numericategoric(df)

TotalNumericalData = 2
TotalCategoricalData = 2
Numerical = ['a', 'c']
Categorical = ['b', 'd']


In [ ]:
def threshold_tuning(model, xtest, ytest, thres, fnvalues, fpvalues):
    recall = []
    precision = []
    f1_score = []
    accuracy_score = []
    score = []
    for k in thres:
        y = pd.DataFrame(clf.predict_proba(xtest), columns=['%No','%Yes'])[['%No']]
        y['prediction'] = y['%No'].apply(lambda x: 0 if x > k else 1 )
        recall.append(metrics.recall_score(ytest, y.prediction))
        precision.append(metrics.precision_score(ytest, y.prediction))
        f1_score.append(metrics.f1_score(ytest, y.prediction))
        accuracy_score.append(metrics.accuracy_score(ytest, y.prediction))
        score.append((metrics.confusion_matrix(ytest, y.prediction)[1][0] * fnvalues) + (metrics.confusion_matrix(y_test, y.prediction)[0][1] * fpvalues))
    df = pd.DataFrame([thres,recall,precision,f1_score,accuracy_score,score]).transpose()
    df.columns = ['thresold','recall','precision','f1-score','accuracy-score','score']
    return df